In [ ]:
!pip install -q easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.5 MB/s eta 0:00:00


In [ ]:
!pip install timm

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
from transformers import AutoModelForObjectDetection
from PIL import Image, ImageDraw
import easyocr
import re
import pandas as pd
# from pdf2image import convert_from_path
from itertools import permutations
from torchvision import transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-detection", revision="no_timm").to(device)



images_folder = "/content/drive/My Drive/diplom/images/norm_inv"
output_csv = "/content/drive/My Drive/diplom/results.csv"




In [ ]:
class MaxResize(object):
    def __init__(self, max_size=800):
        self.max_size = max_size

    def __call__(self, image):
        width, height = image.size
        current_max_size = max(width, height)
        scale = self.max_size / current_max_size
        resized_image = image.resize((int(round(scale*width)), int(round(scale*height))))

        return resized_image

detection_transform = transforms.Compose([
    MaxResize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b


# update id2label to include "no object"
id2label = model.config.id2label
id2label[len(model.config.id2label)] = "no object"


def outputs_to_objects(outputs, img_size, id2label):
    m = outputs.logits.softmax(-1).max(-1)
    pred_labels = list(m.indices.detach().cpu().numpy())[0]
    pred_scores = list(m.values.detach().cpu().numpy())[0]
    pred_bboxes = outputs['pred_boxes'].detach().cpu()[0]
    pred_bboxes = [elem.tolist() for elem in rescale_bboxes(pred_bboxes, img_size)]

    objects = []
    for label, score, bbox in zip(pred_labels, pred_scores, pred_bboxes):
        class_label = id2label[int(label)]
        if not class_label == 'no object':
            objects.append({'label': class_label, 'score': float(score),
                            'bbox': [float(elem) for elem in bbox]})

    return objects

def objects_to_crops(img, tokens, objects, class_thresholds, padding=10):
    """
    Process the bounding boxes produced by the table detection model into
    cropped table images and cropped tokens.
    """

    table_crops = []
    for obj in objects:
        if obj['score'] < class_thresholds[obj['label']]:
            continue

        cropped_table = {}

        bbox = obj['bbox']
        bbox = [bbox[0]-padding, bbox[1]-padding, bbox[2]+padding, bbox[3]+padding]

        cropped_img = img.crop(bbox)

        table_tokens = [token for token in tokens if iob(token['bbox'], bbox) >= 0.5]
        for token in table_tokens:
            token['bbox'] = [token['bbox'][0]-bbox[0],
                             token['bbox'][1]-bbox[1],
                             token['bbox'][2]-bbox[0],
                             token['bbox'][3]-bbox[1]]

        # If table is predicted to be rotated, rotate cropped image and tokens/words:
        if obj['label'] == 'table rotated':
            cropped_img = cropped_img.rotate(270, expand=True)
            for token in table_tokens:
                bbox = token['bbox']
                bbox = [cropped_img.size[0]-bbox[3]-1,
                        bbox[0],
                        cropped_img.size[0]-bbox[1]-1,
                        bbox[2]]
                token['bbox'] = bbox

        cropped_table['image'] = cropped_img
        cropped_table['tokens'] = table_tokens

        table_crops.append(cropped_table)

    return table_crops

In [ ]:
device

'cuda'

In [ ]:
# Список месяцев на русском
months = [
    "январь", "февраль", "март", "апрель", "май", "июнь",
    "июль", "август", "сентябрь", "октябрь", "ноябрь", "декабрь"
]

# Функция для проверки наличия месяца в тексте
def contains_month(text):
    for month in months:
        if month in text.lower():
            return True
    return False

# Регулярное выражение для поиска чисел
number_pattern = re.compile(r'\d+[\.,]?\d*')

# Функция для извлечения чисел
def extract_numbers(data):
    numbers = []
    for _, text in data:
        # Игнорируем строки, содержащие месяцы и годы
        if not contains_month(text) and 'г' not in text.lower():
            matches = number_pattern.findall(text)
            for match in matches:
                numbers.append(float(match.replace(',', '.')))
    return numbers

# Функция для поиска наибольшего числа
def find_energy_amount(numbers):
    max_energy = -1

    for energy_value in numbers:
        if energy_value <= 9999 and energy_value > max_energy:
            max_energy = energy_value
    return max_energy

In [ ]:
import math
def calculate_center(coordinates):
    """Calculate the center of a bounding box given its coordinates."""
    x_coords = [point[0] for point in coordinates]
    y_coords = [point[1] for point in coordinates]
    center_x = sum(x_coords) / len(x_coords)
    center_y = sum(y_coords) / len(y_coords)
    return (center_x, center_y)

def euclidean_distance(coord1, coord2):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def find_nearest_value(data, keyword, ignore_value='101528843'):
    """Find the nearest value to the keyword based on coordinates, ignoring a specific value."""
    keyword_center = None
    values = []

    for item in data:
        coordinates, text, _ = item
        text = correct_text(text)
        center = calculate_center(coordinates)

        # Если ключевое слово находится в тексте
        if keyword in text:
            keyword_center = center

            # Проверяем, есть ли значение, которое мы ищем, в той же строке
            matches = re.findall(r'\d{9}', text)
            for match in matches:
                if match != ignore_value:
                    return match

        # Если текст соответствует шаблону значения
        elif re.match(r'^\d{9}$', text):
            if text != ignore_value:
                values.append((center, text))

    if not keyword_center or not values:
        return None

    nearest_value = min(values, key=lambda x: euclidean_distance(keyword_center, x[0]))
    return nearest_value[1]

def correct_text(text):
    """Корректировка текста, замена 'УНН' на 'УНП'."""
    return re.sub(r'УНН', 'УНП', text)
key_word = ['УНП']

In [ ]:
%%time
all_results = []

for filename in os.listdir(images_folder):
    file_path = os.path.join(images_folder, filename)

    if filename.endswith(".jpg") or filename.endswith(".png"):
        image = Image.open(file_path)
    elif filename.endswith(".pdf"):
        images_from_pdf = convert_from_path(file_path)
        base_filename = os.path.splitext(file_path)[0]
        file_path = f'{base_filename}_page1.png'
        images_from_pdf[0].save(file_path, 'PNG')
        image = Image.open(file_path)
    else:
        continue

    reader = easyocr.Reader(['ru'])
    bounds = reader.readtext(file_path)

    df_for_UNP = bounds

    key_word = 'УНП'
    UNP = find_nearest_value(df_for_UNP, key_word)

    pixel_values = detection_transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(pixel_values)

    objects = outputs_to_objects(outputs, image.size, id2label)
    tokens = []

    detection_class_thresholds = {
        "table": 0.5,
        "table rotated": 0.5,
        "no object": 10
    }
    tables_crops = objects_to_crops(image, tokens, objects, detection_class_thresholds, padding=0)

    if tables_crops:
        cropped_table = tables_crops[0]['image'].convert("RGB")
        table_image_path = f'/content/drive/My Drive/diplom/images/table/table_{filename}.png'
        cropped_table.save(table_image_path)

        bounds_table = reader.readtext(table_image_path)
        df_table = [(bound[0], bound[1].replace(' ', '')) for bound in bounds_table]

        numbers = extract_numbers(df_table)
        max_number = find_energy_amount(numbers)

        all_results.append({
            "UNP": UNP,
            "max_number": max_number
        })
    else:
        print(f"No tables detected in {filename}")
        all_results.append({
            "UNP": UNP,
            "max_number": None
        })

df = pd.DataFrame(all_results)
df.to_csv(output_csv, index=False)


No tables detected in А1 П_page1.png
No tables detected in А1_page1.png
CPU times: user 2min 48s, sys: 11.9 s, total: 3min
Wall time: 3min 4s


In [ ]:
pd.options.display.float_format ='{:,.3f}'.format

In [ ]:
df

,UNP,max_number
0,191111848,"8,295.000"
1,None,529.000
2,191634233,NaN
3,291554478,340.000
4,291618137,65.000
5,490323098,"1,796.000"
6,200106183,"2,296.000"
7,800017207,174.000
8,100742827,81.000
9,192298526,304.000



*  УНП находит правильно в 24/25
* в 3х из 28 счетов  не было данных об УНП 
* в 2ух не удалось правильно выделить таблицу
* в 3х счетах из-за формата написания информации "1 234" не видит 1 и в результате получаю 234
* в 2ух из-за плохого качества неверно определило число
* в остальных 21 счетах все определилось корректно
